In [1]:
import os
import json
import cv2

with open("../Grounded-Segment-Anything/detect_results.json", "r") as f:
    data = json.load(f)


In [4]:
files = [x for x in os.listdir('ndh') if 'test' not in x]

max_bbox_areas = []
for file in files:
    path = os.path.join('../transform_format/ndh', file)
    with open(path, 'r') as f:
        ndh_data = json.load(f)
        for ins in ndh_data:
            scan = ins['scan']
            end_panos = ins['end_panos']
            target = ins['target']
            instr_id = ins['instr_id']

            keys = []
            bboxes = []
            scores = []
            for end_pano in end_panos:
                for idx in range(36):
                    key = f'{scan}_{end_pano}_{target}_{idx}'
                    if key in data:
                        cands = data[key]
                        for jdx in range(len(cands)):
                            if data[key][jdx][1].split('(')[0] == target:
                                keys.append(key)
                                bboxes.append(data[key][jdx][0])
                                scores.append(float(data[key][jdx][1].split('(')[1].split(')')[0]))       

            if scores:
                max_idx = scores.index(max(scores))
                max_score = max(scores)
                assert max_score == scores[max_idx]
                max_bbox = bboxes[max_idx]
                max_key = keys[max_idx]

                x1, y1, x2, y2 = max_bbox
                area = (x2-x1)*(y2-y1)
                max_bbox_areas.append(area)

                view_index = max_key.split('_')[-1]
                view_pano = max_key.split('_')[1]
                view_path = f'/mnt/data/haodong/envedit/views_img/{scan}/{view_pano}/{view_index}.jpg'
                assert os.path.exists(view_path)
                image = cv2.imread(view_path)

                Height, Width = image.shape[:2]
                # Expand the bbox for x pixels
                expand = 30
                x1 = int(max(0, x1-expand))
                y1 = int(max(0, y1-expand))
                x2 = int(min(Width, x2+expand))
                y2 = int(min(Height, y2+expand))

                # crop the image
                crop_img = image[y1:y2, x1:x2]

                # save the image
                save_path = os.path.join("/mnt/data/haodong/UTSD", scan, str(instr_id), f"{target}.jpg")
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                cv2.imwrite(save_path, crop_img)
                cv2.imwrite(save_path.replace('.jpg', '_full.jpg'), image)



# # Draw the distribution of bbox areas
# import matplotlib.pyplot as plt
# import numpy as np
# plt.hist(max_bbox_areas, bins=100)
# plt.title("Histogram of bbox areas")
# plt.xlabel("Area")
# plt.ylabel("Frequency")
# plt.show()


In [ ]:
files = [x for x in os.listdir('ndh') if 'test' not in x]

for file in files:
    path = os.path.join('../transform_format/ndh', file)
    with open(path, 'r') as f:
        ndh_data = json.load(f)
        for ins in ndh_data:
            scan = ins['scan']
            end_panos = ins['end_panos']
            target = ins['target']
            instr_id = ins['instr_id']

            save_path = os.path.join("/mnt/data/haodong/UTSD", scan, str(instr_id))
            assert os.path.exists(save_path)
            assert len(os.listdir(save_path)) == 2